<a href="https://colab.research.google.com/github/bagustris/ravdess_song/blob/main/ravdess_song_sd_fc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAVDESS Song Emotion Recognition
Including feature extraction process, model: Dense  
to be run from Google Colab

In [1]:
import glob
import os
import librosa # install terpisah. bukan library bawaan dari python
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf # install terpisah. bukan library bawaan dari python

In [2]:
#!git clone https://github.com/bagustris/ravdess_song.git

In [3]:
#cd ravdess_song/

In [4]:
pwd

'/home/fafa/Documents/ravdess_song'

In [5]:
# list all files in the dataset
# library glob tidak mengurutkan, sehingga butuh sort setelahnya
data_path = 'archive'
files = glob.glob(os.path.join(data_path + '/*/', '*.wav'))
files.sort()

In [6]:
# check by showing the first file
files[0]

'archive/Actor_01/03-02-01-01-01-01-01.wav'

In [7]:
# function to extract feature
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name, sr=None)
    stft = np.abs(librosa.stft(X))
    mfcc = np.mean(librosa.feature.mfcc(
        y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
    mfcc_std = np.std(librosa.feature.mfcc(
        y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(
        S=stft, sr=sample_rate).T, axis=0)
    chroma_std = np.std(librosa.feature.chroma_stft(
        S=stft, sr=sample_rate).T, axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
    mel_std = np.std(librosa.feature.melspectrogram(
        X, sr=sample_rate).T, axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(
        S=stft, sr=sample_rate).T, axis=0)
    contrast_std = np.std(librosa.feature.spectral_contrast(
        S=stft, sr=sample_rate).T, axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(
        y=librosa.effects.harmonic(X), sr=sample_rate).T, axis=0)
    tonnetz_std = np.std(librosa.feature.tonnetz(
        y=librosa.effects.harmonic(X), sr=sample_rate).T, axis=0)
    return (mfcc, chroma, mel, contrast, tonnetz,
            mfcc_std, chroma_std, mel_std, contrast_std, tonnetz_std)

In [8]:
# create empty list to store features and labels
feat = []
lab = []

In [9]:
# iterate over all files
for file in files:
    print("processing ...", file)
    feat_i = np.hstack(extract_feature(file))
    lab_i = os.path.basename(file).split('-')[2]
    feat.append(feat_i)
    lab.append(int(lab_i)-1)  # make labels start from 0

processing ... archive/Actor_01/03-02-01-01-01-01-01.wav
processing ... archive/Actor_01/03-02-01-01-01-02-01.wav
processing ... archive/Actor_01/03-02-01-01-02-01-01.wav
processing ... archive/Actor_01/03-02-01-01-02-02-01.wav
processing ... archive/Actor_01/03-02-02-01-01-01-01.wav
processing ... archive/Actor_01/03-02-02-01-01-02-01.wav
processing ... archive/Actor_01/03-02-02-01-02-01-01.wav
processing ... archive/Actor_01/03-02-02-01-02-02-01.wav
processing ... archive/Actor_01/03-02-02-02-01-01-01.wav
processing ... archive/Actor_01/03-02-02-02-01-02-01.wav
processing ... archive/Actor_01/03-02-02-02-02-01-01.wav
processing ... archive/Actor_01/03-02-02-02-02-02-01.wav
processing ... archive/Actor_01/03-02-03-01-01-01-01.wav
processing ... archive/Actor_01/03-02-03-01-01-02-01.wav
processing ... archive/Actor_01/03-02-03-01-02-01-01.wav
processing ... archive/Actor_01/03-02-03-01-02-02-01.wav
processing ... archive/Actor_01/03-02-03-02-01-01-01.wav
processing ... archive/Actor_01

In [17]:
len(feat)
len(lab)

1012

In [18]:
# assign hsf to X, lab to y; reshape X for LSTM
X = np.array(feat)
y = np.array(lab)

In [19]:
# split into train and test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [20]:
# functio to build FCN 
def model_dense():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.BatchNormalization(axis=-1,
              input_shape=(x_train.shape[1:])))
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(6, activation='softmax'))

    # compile model: set loss, optimizer, metric
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    return model


In [21]:
# create the model
model = model_dense()
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 386)               1544      
_________________________________________________________________
dense_4 (Dense)              (None, 256)               99072     
_________________________________________________________________
dense_5 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_6 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 6)                 1542      
Total params: 233,742
Trainable params: 232,970
Non-trainable params: 772
______________________________________________

In [27]:
# train the FC model
hist = model.fit(x_train, y_train, epochs=100, shuffle=True, batch_size=16, validation_split=0.1,)

Epoch 1/100
52/52 [==============================] - 2s 41ms/step - loss: 0.0202 - accuracy: 0.9927 - val_loss: 2.5919e-05 - val_accuracy: 1.0000
Epoch 2/100
52/52 [==============================] - 2s 34ms/step - loss: 0.1829 - accuracy: 0.9560 - val_loss: 0.0026 - val_accuracy: 1.0000
Epoch 3/100
52/52 [==============================] - 2s 34ms/step - loss: 0.0949 - accuracy: 0.9707 - val_loss: 0.0022 - val_accuracy: 1.0000
Epoch 4/100
52/52 [==============================] - 2s 33ms/step - loss: 0.0291 - accuracy: 0.9890 - val_loss: 0.0032 - val_accuracy: 1.0000
Epoch 5/100
52/52 [==============================] - 2s 36ms/step - loss: 0.0143 - accuracy: 0.9939 - val_loss: 5.2522e-04 - val_accuracy: 1.0000
Epoch 6/100
52/52 [==============================] - 2s 37ms/step - loss: 0.0202 - accuracy: 0.9951 - val_loss: 3.2344e-04 - val_accuracy: 1.0000
Epoch 7/100
52/52 [==============================] - 2s 38ms/step - loss: 0.0835 - accuracy: 0.9841 - val_loss: 0.0199 - val_accuracy: 0

KeyboardInterrupt: 

In [23]:
evaluate = model.evaluate(x_test, y_test, batch_size=16)
print("Loss: ", evaluate[0], "--> Accuracy: ", evaluate[1])

7/7 [==============================] - 0s 4ms/step - loss: 0.2185 - accuracy: 0.9412
Loss:  0.21853454411029816 --> Accuracy:  0.9411764740943909


Plot hasil

In [26]:
hist.history['val_accuracy']

KeyError: 'val_accuracy'